In [1]:
import re, requests, json, pandas as pd
from bs4 import BeautifulSoup as bs
from pprint import pprint

with open("states.txt") as states:
    statelist = [state for state in states.read().splitlines()]

In [2]:
ID = 74 # test hospital

def searchDict(dictionary, value, term):
    keys = []
    items = dictionary.items()
    for item  in items:
        if item[1][term] == value:
            keys.append(item[0])
    return keys

def waitTime(hospitalID):
    query_address = "https://api.clockwisemd.com/v1/hospitals/"+str(hospitalID)+"/waits"
    api_call = requests.get(query_address)
    data = json.loads(api_call.content)
    return data['hospital_waits']

def hospitalInfo(hospitalID):
    try:
        link = "https://www.clockwisemd.com/hospitals/"+str(hospitalID)+"/unavailable"
        page = requests.get(link)
        soup = bs(page.content, 'html.parser')
        data = soup.find_all('h4')
        name = data[0].get_text()
        city, etc = data[3].get_text().split(", ")
        state, zipcode = etc.split()
        info = {'name':name,'address':data[1].get_text(),'city':city,'state':state,'zip':zipcode}
        if "DEACTIVATED" in name.upper() or "DEMO" in name.upper():
            return "Inactive clinic"
        else:
            return(info)
    except:
        return None
pprint(hospitalInfo(ID))
pprint(waitTime(ID))

None
{'current_wait': 'N/A',
 'next_available_visit': 'N/A',
 'queue_length': 0,
 'queue_total': 0}


In [3]:
master = {}
for i in range(1,5030):
    info = hospitalInfo(i)
    if type(info) is dict:
        master[info['name']] = {'id':i,'address':info['address'],'city':info['city'],'state':info['state'],'zip':info['zip']}
        print("", end="\rHospital ID #{} added.".format(i))

Hospital ID #5029 added.

In [8]:
organized = {}
for state in statelist:
    statezips = []
    for hospital, info in master.items():
        if info['state'] == state and info['zip'] not in statezips:
            statezips.append(info['zip'])
    organized[state] = {}
    for zipcode in statezips:
        inzip = {}
        for hospital, info in master.items():
            if info['zip'] == zipcode:
                inzip[hospital] = master[hospital]
        organized[state][zipcode] = inzip

In [9]:
jsondata = json.dumps(organized, sort_keys=True, indent=3)
with open('hospitals.json','w+') as file:
    file.write(jsondata)